In [73]:
import requests as rqs
import bs4 
import pandas as pd 
import json 
from imp import reload
import logging
reload(logging)
logging.basicConfig(filename='scrape_ow.log', level=logging.INFO, format='%(asctime)s %(message)s')

KAPI= "f092715753d229bc10273e794ecdfbad"
useOneCall=False

#values_url = "https://api.openweathermap.org/data/2.5/weather?q=Macau&appid="+KAPI
if useOneCall:
    ##request through onecall 
    urlbase = ["https://api.openweathermap.org/data/2.5/onecall?lat=","","&lon=","","&exclude=minutely,hourly,daily&appid=" + KAPI]
else:
    ## request through weather (less limitations)
    urlbase = ["http://api.openweathermap.org/data/2.5/weather?lat=","","&lon=","","&appid="+KAPI]

df=pd.DataFrame(columns=['station_uid','lon','lat','dt', 'sunrise','sunset','temp', 'feels_like','pressure','humidity',
                        'dew_point', 'uvi', 'clouds', 'visibility', 'wind_speed', 'wind_deg'])


In [74]:
def dico_from_onecall( ds):
    dico = {'lon':ds['coord']['lon'],'lat':ds['coord']['lat'], 'weather':ds['current']['weather'][0]['main']}
    ds['current'].pop('weather')
    for k in ds['current'].keys():
        dico[k] = ds['current'][k]
        #print(dico)    
    return dico

def dico_from_weather(ds):
        """   example of ds:
        {"coord":{"lon":139,"lat":35},
         "weather":[{"id":500,"main":"Rain","description":"light rain","icon":"10d"}],"base":"stations",
         "main":{"temp":288.71,"feels_like":287.73,"temp_min":288.71,"temp_max":288.71,"pressure":1001,"humidity":92},
         "visibility":3965,"wind":{"speed":3.34,"deg":222},"rain":{"1h":0.18},"clouds":{"all":94},"dt":1614662949,
         "sys":{"type":3,"id":2019346,"country":"JP","sunrise":1614633167,"sunset":1614674398},"timezone":32400,"id":1851632,"name":"Shuzenji","cod":200}
        """
       
        dico = {'lon':ds['coord']['lon'],'lat':ds['coord']['lat'], 
                'weather':ds['weather'][0]['main'], 'weather_desc':ds['weather'][0]['description']}
        for k in ds['main'].keys():
            dico[k] = ds['main'][k]
        dico['visibility']= ds['visibility']
        dico['wind_speed']= ds['wind']['speed']
        dico['wind_deg']= ds['wind']['deg']
        dico['rain']= ds['rain'] if 'rain' in list(ds.keys()) else ''
        dico['clouds']= ds['clouds']
        dico['dt']= ds['dt']
        dico['country']= ds['sys']['country'] if 'country' in list(ds['sys'].keys()) else ''
        dico['sunrise']= ds['sys']['sunrise']
        dico['sunset']= ds['sys']['sunset']
        
        return dico



# Read stations measures 

In [75]:
dfstations=pd.read_csv("station_list.csv",header=0)

for i,s in dfstations.iterrows():
    #print(s)
    urlbase[1],urlbase[3]=str(s.lat),str(s.lon)
    url= ''.join(urlbase)
    rs = rqs.get(url)
    ds = json.loads(rs.text)
    if useOneCall:
        dico = dico_from_onecall(ds)
    else:
        dico = dico_from_weather(ds)
    dico['station_uid']=s.uid
    df=df.append(dico, ignore_index=True) 


df

,station_uid,lon,lat,dt,sunrise,sunset,temp,feels_like,pressure,humidity,...,clouds,visibility,wind_speed,wind_deg,country,rain,temp_max,temp_min,weather,weather_desc
0,9888,113.5817,22.1321,1614666409,1614638776,1614680988,294.52,289.78,1018,60,...,{'all': 0},10000,8.23,360,MO,,295.15,294.15,Clear,clear sky
1,785,113.5740,22.2611,1614666410,1614638783,1614680985,295.01,290.43,1018,60,...,{'all': 0},10000,8.23,360,CN,,295.93,294.15,Clear,clear sky
2,9052,113.5544,22.1253,1614666410,1614638783,1614680994,294.15,289.30,1018,60,...,{'all': 0},10000,8.23,360,MO,,294.15,294.15,Clear,clear sky
3,1540,113.5767,22.2707,1614666410,1614638782,1614680984,295.01,290.43,1018,60,...,{'all': 0},10000,8.23,360,CN,,295.93,294.15,Clear,clear sky
4,5373,113.5650,22.1600,1614666410,1614638781,1614680991,294.53,289.80,1018,60,...,{'all': 0},10000,8.23,360,MO,,295.15,294.15,Clear,clear sky
5,786,113.4950,22.2294,1614666410,1614638801,1614681005,294.95,290.35,1018,60,...,{'all': 0},10000,8.23,360,MO,,295.93,294.15,Clear,clear sky
6,5372,113.5447,22.1958,1614666410,1614638787,1614680994,294.15,289.30,1018,60,...,{'all': 0},10000,8.23,360,MO,,294.15,294.15,Clear,clear sky
7,5374,113.5428,22.2139,1614666410,1614638789,1614680994,294.57,289.85,1018,60,...,{'all': 0},10000,8.23,360,MO,,295.15,294.15,Clear,clear sky
8,5375,113.5546,22.1581,1614666410,1614638784,1614680993,294.15,289.30,1018,60,...,{'all': 0},10000,8.23,360,MO,,294.15,294.15,Clear,clear sky


# Read weather grid further from stations

In [76]:
#set grid 
latmax, lonmax= 31.266224, 122.238532  #shanghai
latmin, lonmin=  19.258517, 104.907299  #vietnam
nblats, nblons=5,5
steplat=(latmax-latmin)/nblats
steplon=(lonmax-lonmin)/nblons

dfgrid = pd.DataFrame([{'lat':latmin+i*steplat, 'lon':lonmin+j*steplon} for i in range(nblats) for j in range(nblons)])

In [77]:
import time

for i,s in dfgrid.iterrows():
    #print(s)
    time.sleep(1)  # limitation 60 requests per minute : cf https://openweathermap.org/price
    urlbase[1],urlbase[3]=str(s.lat),str(s.lon)
    url= ''.join(urlbase)
    rs = rqs.get(url)
    ds = json.loads(rs.text)
    
    if useOneCall:
        dico = dico_from_onecall(ds)
    else:
        dico = dico_from_weather(ds)
    dico['station_uid']=0
    
    #print(dico)
    df=df.append(dico, ignore_index=True) 

df

,station_uid,lon,lat,dt,sunrise,sunset,temp,feels_like,pressure,humidity,...,wind_speed,wind_deg,country,rain,temp_max,temp_min,weather,weather_desc,grnd_level,sea_level
0,9888,113.5817,22.1321,1614666409,1614638776,1614680988,294.52,289.78,1018,60,...,8.23,360,MO,,295.15,294.15,Clear,clear sky,NaN,NaN
1,785,113.5740,22.2611,1614666410,1614638783,1614680985,295.01,290.43,1018,60,...,8.23,360,CN,,295.93,294.15,Clear,clear sky,NaN,NaN
2,9052,113.5544,22.1253,1614666410,1614638783,1614680994,294.15,289.30,1018,60,...,8.23,360,MO,,294.15,294.15,Clear,clear sky,NaN,NaN
3,1540,113.5767,22.2707,1614666410,1614638782,1614680984,295.01,290.43,1018,60,...,8.23,360,CN,,295.93,294.15,Clear,clear sky,NaN,NaN
4,5373,113.5650,22.1600,1614666410,1614638781,1614680991,294.53,289.80,1018,60,...,8.23,360,MO,,295.15,294.15,Clear,clear sky,NaN,NaN
5,786,113.4950,22.2294,1614666410,1614638801,1614681005,294.95,290.35,1018,60,...,8.23,360,MO,,295.93,294.15,Clear,clear sky,NaN,NaN
6,5372,113.5447,22.1958,1614666410,1614638787,1614680994,294.15,289.30,1018,60,...,8.23,360,MO,,294.15,294.15,Clear,clear sky,NaN,NaN
7,5374,113.5428,22.2139,1614666410,1614638789,1614680994,294.57,289.85,1018,60,...,8.23,360,MO,,295.15,294.15,Clear,clear sky,NaN,NaN
8,5375,113.5546,22.1581,1614666410,1614638784,1614680993,294.15,289.30,1018,60,...,8.23,360,MO,,294.15,294.15,Clear,clear sky,NaN,NaN
9,0,104.9073,19.2585,1614666434,1614640761,1614683166,289.61,290.33,1017,92,...,1.36,129,VN,,289.61,289.61,Clouds,overcast clouds,935.0,1017.0


# save in csv

In [80]:
fn='scraped_ow_data' +  ("_weather" if useOneCall==False else "_onecall" ) + '.csv'
df.to_csv(fn, mode='a', header=False,index=False)  # record each url one after the other ... 
